### _LCEL - LANGCHAIN EXPRESSION LANGUAGE_ ###

In [2]:
#Import libraries
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#Setting environmental variables
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')


In [4]:
#creating an Object of Chatgroq model
from langchain_groq import ChatGroq
model = ChatGroq(model ='gemma2-9b-it',groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f629a0071c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f629a030670>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
messages1 = [
    ("system", "You are a helpful translator. Translate the user sentence to hindi."),
    ("human", "I love programming."),
]
response = model.invoke(messages)
print(response.content)

नमस्ते, शुभप्रभात 



In [14]:
#A list of instructions as Human and System message
from langchain_core.messages import HumanMessage,SystemMessage
messages2 =[SystemMessage(content='Translate the given text from English to Hindi'),
           HumanMessage(content='Hello,Good Morning')]

In [17]:
result = model.invoke(messages2)
result

AIMessage(content='नमस्ते, सुप्रभात \n\n\nLet me know if you need any other words or phrases translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 21, 'total_tokens': 48, 'completion_time': 0.049090909, 'prompt_time': 0.002117771, 'queue_time': 0.23440011200000002, 'total_time': 0.05120868}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f352df67-c9f1-4b0d-b735-33cb5e5a3321-0', usage_metadata={'input_tokens': 21, 'output_tokens': 27, 'total_tokens': 48})

In [12]:
#Parsing the outputs  without metadata
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'नमस्ते, शुभ प्रभात \n'

In [23]:
#Using LCEL to chain the components
chain =model|parser
print(chain.invoke(messages1))

मुझे प्रोग्रामिंग बहुत पसंद है। 



In [21]:
print(chain.invoke(messages2))

नमस्ते, सुप्रभात् 



### _Method2 - Using ChatPromptTemplate and LCEL_ ###

In [27]:
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the given text to {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),
     ("user","{text}")]
)

In [32]:
result = prompt.invoke({'language':'French',
                        'text':'Hello'})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the given text to French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [33]:
result.to_messages()

[SystemMessage(content='Translate the given text to French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [34]:
model.invoke(result)

AIMessage(content='Bonjour  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 16, 'total_tokens': 21, 'completion_time': 0.009090909, 'prompt_time': 0.001890656, 'queue_time': 0.231712308, 'total_time': 0.010981565}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e435e75-ab5b-425e-8dc2-b11481770099-0', usage_metadata={'input_tokens': 16, 'output_tokens': 5, 'total_tokens': 21})

In [36]:
#Chaining diff components using LCEL
chain = prompt|model|parser
chain.invoke({'language':'French',
              'text':'Hello'})

'Bonjour \n'